# Detect AI vs Human generated images
In this notebook, I aim to build a convnet architecture for detecting real images from fake ones.

In [1]:
# Libraries used in this notebook
import kagglehub
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import Xception
from keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [2]:
# Download latest version
data_path = kagglehub.dataset_download("alessandrasala79/ai-vs-human-generated-dataset")

In [3]:
# Csv with references to the images
train_data = pd.read_csv(data_path + "/" + "train.csv", index_col=0)
test_data = pd.read_csv(data_path + "/" + "test.csv")

In [4]:
train_data.head()

,file_name,label
0,train_data/a6dcb93f596a43249135678dfcfc17ea.jpg,1
1,train_data/041be3153810433ab146bc97d5af505c.jpg,0
2,train_data/615df26ce9494e5db2f70e57ce7a3a4f.jpg,1
3,train_data/8542fe161d9147be8e835e50c0de39cd.jpg,0
4,train_data/5d81fa12bc3b4cea8c94a6700a477cf2.jpg,1


In [5]:
# Creates a specific paths to each image
train_full_path = train_data["file_name"].apply(lambda image_path: data_path + "/" + image_path).values
test_full_path = test_data["id"].apply(lambda image_path: data_path + "/" + image_path).values

# Labels for training data
train_labels = train_data.loc[:,"label"].values

In [6]:
# Function to load and preprocess an image
def load_and_preprocess_image(path, label=None):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)  # Ensure RGB format
    image = tf.image.resize(image, [224, 224])  # Resize to ResNet50 input size
    image = preprocess_input(image)  # Apply ResNet50 preprocessing
    return image, label

# Create a TensorFlow dataset from image paths
train_dataset = tf.data.Dataset.from_tensor_slices((train_full_path, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_full_path))

# Shuffle train dataset
buffer_size = len(train_dataset)  # Set buffer size to dataset size for perfect shuffling
train_dataset = train_dataset.shuffle(buffer_size, reshuffle_each_iteration=True)

# Map the dataset to load and preprocess images
train_dataset = train_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)

# Batch the dataset (optional)
batch_size = 32
train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [10]:
base_model = keras.applications.resnet50.ResNet50(weights="imagenet", include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(2, activation="softmax")(avg)
model = keras.Model(inputs=base_model.input, outputs=output)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [11]:
for layer in base_model.layers:
    layer.trainable = False

In [12]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.2, 
                                                          decay_steps=10000, 
                                                          decay_rate=0.96, 
                                                          staircase=True)

optimizer = keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(train_dataset, epochs=10)


Epoch 1/10
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 249s 95ms/step - accuracy: 0.9175 - loss: 6.6115
Epoch 2/10
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 224s 90ms/step - accuracy: 0.9523 - loss: 4.6990
Epoch 3/10
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 223s 89ms/step - accuracy: 0.9576 - loss: 4.1168
Epoch 4/10
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 223s 89ms/step - accuracy: 0.9563 - loss: 4.3551
Epoch 5/10
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 223s 89ms/step - accuracy: 0.9608 - loss: 4.2822
Epoch 6/10
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 223s 89ms/step - accuracy: 0.9625 - loss: 3.9525
Epoch 7/10
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 223s 89ms/step - accuracy: 0.9628 - loss: 3.6003
Epoch 8/10
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 223s 89ms/step - accuracy: 0.9604 - loss: 4.1451
Epoch 9/10
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 223s 89ms/step - accuracy: 0.9636 - loss: 3.7494
Epoch 10/10
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 224s 89ms/step - accuracy: 0.9647 - loss: 3.4793


In [13]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.01, 
                                                          decay_steps=10000, 
                                                          decay_rate=0.96, 
                                                          staircase=True)
for layer in base_model.layers:
    layer.trainable = True
optimizer = keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(train_dataset, epochs=10)

Epoch 1/10
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 793s 302ms/step - accuracy: 0.7154 - loss: 17.7092
Epoch 2/10
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 738s 295ms/step - accuracy: 0.8271 - loss: 1.5609
Epoch 3/10
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 739s 295ms/step - accuracy: 0.8433 - loss: 0.9423
Epoch 4/10
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 738s 295ms/step - accuracy: 0.8307 - loss: 0.9563
Epoch 5/10
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 740s 296ms/step - accuracy: 0.8357 - loss: 1.6537
Epoch 6/10
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 737s 295ms/step - accuracy: 0.8460 - loss: 1.0599
Epoch 7/10
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 735s 294ms/step - accuracy: 0.8476 - loss: 0.6713
Epoch 8/10
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 739s 296ms/step - accuracy: 0.8554 - loss: 0.6088
Epoch 9/10
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 740s 296ms/step - accuracy: 0.8586 - loss: 0.6294
Epoch 10/10
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 739s 296ms/step - accuracy: 0.8598 - loss: 0.5765


In [14]:
predictions = model.predict(test_dataset)

174/174 ━━━━━━━━━━━━━━━━━━━━ 62s 337ms/step


In [19]:
class_labels = (predictions > 0.5).astype(int)

In [49]:
ar = []
for i in predictions:
    if i[0] > i[1]:
        ar.append(1)
    else:
        ar.append(0)

In [50]:
test_data["label"] = ar

In [51]:
test_data.to_csv("predictions.csv", index=False)

In [53]:
model.save("/kaggle/working/ai-vs-human-images.h5")
